In [5]:
import pandas as pd
import boto3
import json
from dotenv import load_dotenv

In [ ]:
def load():
    path = '../data/credit_risk_imputed.csv'
    data = pd.read_excel(path)
    
    print(data.info())

    return data

In [ ]:
def generate_prompt(row):
    return f"""Profile Summary:
Age: {row['Age']}, Sex: {row['Sex']}, Job: {row['Job']}, 
Housing: {row['Housing']}, Savings account: {row['Saving accounts']}, 
Checking account: {row['Checking account']}, Credit amount: {row['Credit amount']}, 
Loan duration (months): {row['Duration']}, Purpose: {row['Purpose']}.
Generate a brief description of this person's credit risk.
"""

In [ ]:
bedrock_runtime = boto3.client('bedrock-runtime')

def obtener_descripcion(prompt):
    response = bedrock_runtime.invoke_model(
        modelId='anthropic.claude-3-sonnet-20240229-v1:0',  # o llama, titan, etc.
        contentType='application/json',
        body=json.dumps({
            "prompt": prompt,
            "max_tokens": 100
        })
    )
    return json.loads(response['body'].read())['completion']


In [ ]:
def main():
    load_dotenv()
    data = load()
    data["prompt"] = data.apply(generate_prompt, axis=1)
    data["description"] = data["prompt"].apply(obtener_descripcion)
    data.to_csv("credit_risk_with_descriptions.csv", index=False)

In [ ]:
main()